# A real-world example and production mindset

The motivation behind this project is to demonstrate what I had to do for a real-world analysis and how it was to develop with a production mindset. At a high level, this is mostly prototyping with some final observations that can lead to a production pipelines. I will also attempt to highlight the business cases for each part of the analysis, because at the end of the day data science and analytics exists to find insights that drive business decisions.

<p>
An old acquaintance approached me regarding my foray into data science and analytics. After some discussion, I was requested to assist with some historical data analysis and thus requested if I could showcase a portion of this work. It was agreed upon that I may use some of the work, so long as any real names and operation specifics would be left out or obfuscated.
</p>

<p>
For some context, the business deals with content delivery apps that rate interactions with various service providers. 
</p>

## 1. - Transactional data and user interactions
In the first section, we look at some orders that pass through the backend and explore the various interactions of users and business partners. From a business perspective, this is useful in determining information such as peak times and other events that lead to increased demand, high value clients and potential allocation of business resources.

Let's start with the usual imports for data manipulation, then proceed to read in the data and examine the head and tail.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('datasets/dataset1.csv')
df1.head()

,Date,Time,User,Partner,Status,Transaction ($),Category,Sequence Time (min:sec)
0,8/30/2015,7:22 PM,192749,RTAEBO,ACCEPTED,11.30,E,0:14
1,8/30/2015,7:47 PM,697640,G43YQ1,REJECTED,8.60,C,10:18
2,8/30/2015,7:13 PM,192749,5I5LAT,REJECTED,4.19,E,0:36
3,8/30/2015,7:11 PM,192749,5I5LAT,REJECTED,4.19,E,0:23
4,8/30/2015,7:16 PM,192749,5I5LAT,REJECTED,4.19,E,0:17


In [3]:
df1.tail()

,Date,Time,User,Partner,Status,Transaction ($),Category,Sequence Time (min:sec)
3834,9/3/2015,8:32 AM,172171,4E0GZ9,ACCEPTED,1.94,C,1:04
3835,9/3/2015,7:40 AM,987192,4E0GZ9,CANCELLED,1.94,C,NaN
3836,9/3/2015,7:32 AM,844451,KOZM4L,ACCEPTED,1.58,C,0:25
3837,9/3/2015,8:10 AM,801690,UK1GEQ,ACCEPTED,1.47,C,1:18
3838,9/3/2015,9:14 AM,619584,UK1GEQ,ACCEPTED,1.47,C,0:18


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839 entries, 0 to 3838
Data columns (total 8 columns):
Date                       3839 non-null object
Time                       3839 non-null object
User                       3839 non-null int64
Partner                    3839 non-null object
Status                     3839 non-null object
Transaction ($)            3839 non-null float64
Category                   3839 non-null object
Sequence Time (min:sec)    3822 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 240.0+ KB


Observing that there are some null values, it is a good idea to explore where they occur and attempt to determine why.

In [5]:
df1[df1.isnull().any(axis=1)]

,Date,Time,User,Partner,Status,Transaction ($),Category,Sequence Time (min:sec)
270,8/31/2015,7:02 AM,376269,1HFZX3,CANCELLED,12.43,C,NaN
317,8/31/2015,11:54 AM,802368,66CCIF,CANCELLED,11.79,C,NaN
406,8/31/2015,11:36 AM,375782,KIAD9J,CANCELLED,10.51,C,NaN
724,8/31/2015,11:44 AM,656236,TENC59,CANCELLED,6.22,C,NaN
887,8/31/2015,10:45 PM,852434,O6S3BJ,CANCELLED,2.63,B,NaN
1445,9/1/2015,6:25 PM,728136,KUMYVP,CANCELLED,10.11,C,NaN
1559,9/1/2015,2:22 PM,307389,KQ143B,CANCELLED,8.48,C,NaN
1652,9/1/2015,12:11 PM,703945,CICECE,CANCELLED,6.46,C,NaN
1681,9/1/2015,9:04 AM,116929,59GUWG,CANCELLED,5.38,C,NaN
1789,9/1/2015,1:09 PM,237407,CICECE,CANCELLED,3.26,C,NaN


The NaN values in the column "Sequence Time" seem to occur because they correspond with cancelled transactions, so these values will be left as is. From a business standpoint, it might be beneficial to correlate this with other data and understand why a transaction was cancelled, but that was beyond the scope of this analysis.

### 1.1 - Determining how many unique business partners received orders on a particular date
This was fairly straightforward as the data frame takes a single date (Sep. 3, 2015) slice, followed by finding the unique count of the "Partner" column.

In [6]:
df1[df1['Date']=='9/3/2015']['Partner'].nunique()

106

### 1.2 - Finding the highest number of successful transactions on a single day with a given category
To determine this, a conditional slice of Status and Category is used followed by a grouping by date and size aggregation. Applying the max function to all this yields the highest value and which day it occurs on.

In [7]:
df1[(df1['Status'] == 'ACCEPTED') & 
    (df1['Category'] == 'C')].groupby('Date').size().reset_index(name='counts').max()

Date      9/3/2015
counts         737
dtype: object

### 1.3 - A useful point for marketing strategy: high spending customers
For this, the highest amount of money a user spent during the time period of the dataset was aggregated from all accepted orders.
This approach is similar to the one above, however due the users ID being numerical sequence, ```reset_index()``` cannot be used to return a pair as above. 

(As an aside, if ```rest_index()``` is used and ```max()``` is called, it will create a non-index "user" column separately and return two maximum values that do not relate.)

In [8]:
df1[(df1['Status'] == 'ACCEPTED')].groupby('User').sum().max()

Transaction ($)    293.12
dtype: float64

To solve this is a more readable manner, a new data frame is created which slices the maximum value and return both the user number and associated transaction sum. 

(Note: While this is fine for such a relatively small data set such as this, it may not scale well to very large sets in which case more computationally efficient methods should be applied such as the “query” method. See documentation [here]( https://pandas.pydata.org/pandas-docs/stable/indexing.html#performance-of-query))

In [9]:
high_value_user = df1[(df1['Status']=='ACCEPTED')].groupby('User').sum()
high_value_user[high_value_user['Transaction ($)'] == high_value_user['Transaction ($)'].max()]

,Transaction ($)
User,
145158,293.12


### 1.4 - Transaction times and business response
Based on historical trends unique to this business's operations, a response time of under 1 minute is considered ideal, thus to find the times, the data frame can be sliced using categories relating to a particular day and having a response time less than a minute. Here, a particular day (Tuesday, Sep. 1, 2015) was chosen to determine what pecentage of all transactions on that day were faster than a minute.

Recalling from the ```df1.info()``` method earlier, there were some NaN values in the "Sequence Time" column, thus only non-null values will be considered, which means filtering out transactions that were cancelled.

First, the total number of transactions on that day are calculated.

In [10]:
len(df1[(df1['Date'] == '9/1/2015') & (df1['Status'] != 'CANCELLED')])

935

This is followed by calculating all the transactions less than the ideal response time.

In [11]:
len(df1[(df1['Date'] == '9/1/2015') & (df1['Sequence Time (min:sec)']<'1:00')])

632

However, it can all be determined in a (slightly messy) single line.

In [12]:
len(df1[(df1['Date'] == '9/1/2015') &
        (df1['Sequence Time (min:sec)']<'1:00')]) / len(df1[(df1['Date'] == '9/1/2015') &
                                                            (df1['Status'] != 'CANCELLED')]) * 100

67.59358288770053

## 2. - Service improvement and analysis

The second part of the analysis deals with analyzing user feedback about business partners. This provides both users and partners to determine which operations work well and which can use improvement.

To begin, the second dataset is read in and the head and tail are examined.

In [13]:
df2 = pd.read_csv('datasets/dataset2.csv')
df2.head()

,Date,Partner,Score,UX,Transaction Experience,Process Quality
0,11/20/2015,UJZDE8,5,Good,Good,Good
1,11/20/2015,UJZDE8,5,Good,Good,Good
2,11/20/2015,UJZDE8,5,Good,Good,Good
3,11/20/2015,UJZDE8,3,Improving,Good,Good
4,11/29/2015,GXD6NC,5,Good,Good,Good


In [14]:
df2.tail()

,Date,Partner,Score,UX,Transaction Experience,Process Quality
636,11/17/2015,XO55ZB,5,Good,Good,Good
637,11/20/2015,XO55ZB,5,Good,Good,Good
638,11/24/2015,XO55ZB,5,Good,Good,Good
639,11/26/2015,XO55ZB,5,Improving,Good,Good
640,11/27/2015,XO55ZB,5,Good,Good,Good


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 6 columns):
Date                      641 non-null object
Partner                   641 non-null object
Score                     641 non-null int64
UX                        641 non-null object
Transaction Experience    641 non-null object
Process Quality           641 non-null object
dtypes: int64(1), object(5)
memory usage: 30.1+ KB


### 2.1 - The average rating for business partners during a given time frame
It is useful to know how business partners score during certain times. This can help improve certain business functions during holiday rushes or other such events. 

In this case, the week before Black Friday 2015 is examined (Nov. 16-20) and the average rating is calculated by slicing, grouping and then applying a mean function to the mean aggregation (i.e. mean of the mean).

In [16]:
df2[(df2['Date']>='11/16/2015') & (df2['Date']<='11/20/2015')].groupby('Partner')['Score'].mean().mean()

4.42895168956875

### 2.2 - A case study of levelling up

This was the bulk of the analysis for this particular business. The company has a policy of promoting well performing business partners to a "Gold" rating that has certain benefits and privileges. In order for a business partner to achieve this special status, it must achieve the following criteria:

- Have a minimum of 10 user feedback instances
- Have an average score of at least 4.0
- Have a minimum of "Good" in across all categories for 90% of feedback

To start this analysis, the user feedback is counted and only those partners with more than 10 feedback instances are selected.

In [17]:
df2_count = pd.DataFrame(df2.groupby('Partner')['Date'].count())
df2_feedback_threshold = df2_count[df2_count['Date'] >= 10].index.values.tolist()
df2_feedback_threshold

['64P2G1',
 '7Q9M2I',
 '886VOM',
 '91DHOD',
 '9EDN58',
 'FJGVQ4',
 'GXBENY',
 'HYM4L1',
 'J08XUI',
 'J4WJ99',
 'KKIBJ3',
 'MEN2KH',
 'OGZ5KO',
 'P1ENTH',
 'P5QA3E',
 'Q7XKWO',
 'RGN5S7',
 'S6PUQ8',
 'TIA4D5',
 'TT7NS2',
 'XO55ZB',
 'ZZG4ZD']

Next, the score threshold is used to filter out partners that do not meet the minimum. The data frame is first sliced for some visual EDA.

In [18]:
df2_score_threshold = df2[df2['Partner'].isin(df2_feedback_threshold)]
df2_score_threshold = pd.DataFrame(df2_score_threshold.groupby('Partner')['Score'].mean())
df2_score_threshold[df2_score_threshold['Score'] >= 4.0]

,Score
Partner,
64P2G1,4.440000
7Q9M2I,5.000000
886VOM,4.352941
91DHOD,4.466667
9EDN58,4.904762
FJGVQ4,4.400000
GXBENY,4.800000
J08XUI,4.916667
J4WJ99,5.000000


After verification, a list of values containing the partner ID is generated.

In [19]:
df2_score_threshold = df2_score_threshold[df2_score_threshold['Score'] >= 4.0].index.values.tolist()
df2_score_threshold

['64P2G1',
 '7Q9M2I',
 '886VOM',
 '91DHOD',
 '9EDN58',
 'FJGVQ4',
 'GXBENY',
 'J08XUI',
 'J4WJ99',
 'KKIBJ3',
 'MEN2KH',
 'OGZ5KO',
 'P1ENTH',
 'P5QA3E',
 'Q7XKWO',
 'RGN5S7',
 'TIA4D5',
 'TT7NS2',
 'XO55ZB',
 'ZZG4ZD']

The final filter will be to determine which business partners meet the criterion of more than 90% good feedback across all categories.

In [20]:
df2_categorical_threshold = df2[df2['Partner'].isin(df2_score_threshold)]
df2_gold_possible = df2_categorical_threshold[(df2_categorical_threshold['UX']=='Good') &
                                              (df2_categorical_threshold['Transaction Experience']=='Good') &
                                              (df2_categorical_threshold['Process Quality']=='Good')].groupby('Partner').size() /\
                                               df2_categorical_threshold.groupby(['Partner'])[['UX',
                                                                                               'Transaction Experience',
                                                                                               'Process Quality']].size() * 100
print(df2_gold_possible[df2_gold_possible >= 90.0])
print("\nTotal partners with 90%+ Good ratings: ", sum(df2_gold_possible >= 90.0))

Partner
7Q9M2I    100.000000
91DHOD     93.333333
9EDN58    100.000000
J08XUI    100.000000
J4WJ99     92.307692
KKIBJ3     90.000000
dtype: float64

Total partners with 90%+ Good ratings:  6


## 3. - Considerations for a production environment

Generally speaking, when moving to production code one must follow the D.R.Y. principle: Do not Repeat Yourself. Therefore, it is always easier to establish functions for repetitive tasks such as periodic analysis of data that conveys the same information.

In this particular case, if the company wanted to analyze which business partners are ready to qualify for "Gold" on a regular basis, it would be best to define a function that can automatically extract this information rather than explore the same data set every time. This also assumes that data labels and categories will remain unchanged. By abstracting the exploratory analysis above, a sequence of processes can be strung together in a custom function.

To move a bit beyond the base data, I offered a recommendation that the three main criteria have default values so that in the future anybody can grab this function and also use it in a modelling capacity. By varying any or all of the three criteria and comparing with other references, the business can decide if lowering or raising the limits is beneficial or not.

In [21]:
def gold_potential(df, mean_rating=4.0, min_revs=10, min_cat_thresh=90.0):
    """
    This function will filter a DataFrame for the specified criteria:
    - Minimum sample size of reviews, min_revs: default = 10 reviews
    - Avg. user rating, mean_rating: default = 4.0
    - Min percentage of responses "Good" in all categories, min_cat_thresh: default = 90.0%
    """
    criteria = {'Mean rating':mean_rating, 
                'Minimum reviews':min_revs,
                'Minimum categorical %':min_cat_thresh}
    
    # Counting the miniumin specified reviews
    df_feedback_count = pd.DataFrame(df.groupby('Partner')['Date'].count())
    feedback_thresh_list = df_feedback_count[df_feedback_count['Date'] >= min_revs].index.values.tolist()
    
    # Adding minimum reviews as a filter and selecting an average rating greater than the specified minimum
    df_score_thresh = df[df['Partner'].isin(feedback_thresh_list)]
    df_score_thresh = pd.DataFrame(df_score_thresh.groupby('Partner')['Score'].mean())
    score_thresh_list = df_score_thresh[df_score_thresh['Score'] >= mean_rating].index.values.tolist()
    
    # Finding which of the filtered data also has 90% Great in all categories
    df_category_thresh = df[df['Partner'].isin(score_thresh_list)]
    
    categories_cols = ['UX', 'Transaction Experience', 'Process Quality']
    
    categories_check = ((df_category_thresh[categories_cols[0]]=='Good') & 
                        (df_category_thresh[categories_cols[1]]=='Good') &
                        (df_category_thresh[categories_cols[2]]=='Good'))
    
    category_thresh_list = df_category_thresh[categories_check].groupby('Partner').size() /\
                                df_category_thresh.groupby(['Partner'])[categories_cols].size() * 100
    
    return print(criteria, "\n", category_thresh_list[category_thresh_list >= min_cat_thresh],
                 "\nTotal partners ready for gold: ", sum(category_thresh_list >= min_cat_thresh), "\n")

# Exploring various possibilites with the function
gold_potential(df2)
gold_potential(df2, mean_rating=4.5, min_revs=8, min_cat_thresh=80.0)

{'Mean rating': 4.0, 'Minimum reviews': 10, 'Minimum categorical %': 90.0} 
 Partner
7Q9M2I    100.000000
91DHOD     93.333333
9EDN58    100.000000
J08XUI    100.000000
J4WJ99     92.307692
KKIBJ3     90.000000
dtype: float64 
Total partners ready for gold:  6 

{'Mean rating': 4.5, 'Minimum reviews': 8, 'Minimum categorical %': 80.0} 
 Partner
7Q9M2I    100.000000
9EDN58    100.000000
GXBENY     80.000000
J08XUI    100.000000
J4WJ99     92.307692
KKIBJ3     90.000000
P5QA3E     85.714286
Q7XKWO     84.615385
XO55ZB     81.818182
dtype: float64 
Total partners ready for gold:  9 



### 3.1 - A pipeline concept

Being limited in what I can demonstrate due to confidentiality, I chose to abstract at a high level what a potential production pipeline could look like. I'm generalizing here and these steps should adapt to a particular business's data environment and analytical use case.

The data pipeline should contain these important steps:
- Importing data
- Cleaning data
- Exploratory analysis
- Data manipulation

```python

def file_import(files):
    # Read in files with appropriate methods
    
def clean_import(df):
    # Drop or backfill missing values
    
def explore_data(df):
    # Find the most important features and correlations
    # Apply visualizations if required
    
def data_manipulation(df):
    # Combine and aggregate relevant data to gain insights
    return business_intellegence
```

If using machine learning techniques it is the same up to data_manipulation, which becomes data_learning and has these additional steps:
- Training and testing data
- Model training
- Model validation and metrics

```python

def file_import(files):
    # Read in files with appropriate methods
    
def clean_import(df):
    # Drop or backfill missing values
    
def explore_data(df):
    # Find the most important features and correlations
    # Apply visualizations if required

def data_learning(df):
    # Manipulate data and split into train and test sets
    return training_data, test_data

def train_model(training_data, test_data):
    # Train machine learning model
    return model

def validate_model(train_test_split, model):
    # Apply k-fold cross validation
    # Generate metrics
    return metrics
```